In [13]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from math import floor
import pickle
import xgboost as xg
from xgboost.sklearn import XGBRegressor

In [2]:
os.chdir("/Volumes/Data/Adithya/Kaggle/walmart_salesprediction/walmart-recruiting-store-sales-forecasting/")

In [3]:
## load datasets
features = pd.read_csv("datasets/inputdatasets/features.csv")
train = pd.read_csv("datasets/inputdatasets/train.csv")


In [4]:
## initialize variables
seasonality_type = "week"
event_weeks = ["2012-07-06", "2012-04-06"]
level = ["Store", "Dept"]
validation_split = 0.2
model_name = "xgboost"   ### choose from linear, ridge, lasso, random forest, gradient boosting
ridge_range = [0, 20]
lasso_range = [1, 10]



In [6]:
# initialize hyperparameters
grid = {}

if model_name == "random forest":
    ## random forest hyperparameters

    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] # Number of trees in random forest
    max_features = ['auto', 'sqrt'] # Number of features to consider at every split
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Maximum number of levels in tree
    max_depth.append(None)
    min_samples_split = [2 , 5, 10, 50] # Minimum number of samples required to split a node
    min_samples_leaf = [1, 2, 4, 20] # Minimum number of samples required at each leaf node

    # Create the random grid
    grid = {'n_estimators': n_estimators,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
                   }
    
elif model_name == "gradient boosting":
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] # Number of trees in random forest
    learning_rate = np.linspace(start = 0.1, stop = 0.3, num = 11)
    max_features = ['auto', 'sqrt'] # Number of features to consider at every split
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Maximum number of levels in tree
    max_depth.append(None)
    min_samples_split = [2 , 5, 10, 50] # Minimum number of samples required to split a node
    min_samples_leaf = [1, 2, 4, 20] # Minimum number of samples required at each leaf node

    # Create the random grid
    grid = {'n_estimators': n_estimators,
            'learning_rate':learning_rate,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
                   }
    
elif model_name == "xgboost":
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    eta = [0.01, 0.015, 0.025, 0.05, 0.1]
    gamma = [0.05, 0.075, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
    max_depth = [int(x) for x in np.linspace(start = 3, stop = 24, num = 8)]
    min_child_weight = [int(x) for x in np.linspace(start = 1, stop = 9, num = 5)]
    subsample = np.linspace(start = 0.6, stop = 1, num = 5)
    colsample_bytree = np.linspace(start = 0.6, stop = 1, num = 5)
    reg_lambda = [0.01, 0.05, 0.1, 0.5, 1]
    reg_alpha = [0.01, 0.05, 0.1, 0.5, 1]
    
    grid = { 'n_estimators': n_estimators,
            'eta': eta,
            'gamma': gamma,
            'min_child_weight': min_child_weight,
            'max_depth': max_depth,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree,
            'reg_lambda': reg_lambda,
            'reg_alpha': reg_alpha
        
    }
    
# ##
# gbcheck = GradientBoostingRegressor()
# print(gbcheck.get_params())

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'eta': [0.01, 0.015, 0.025, 0.05, 0.1], 'gamma': [0.05, 0.075, 0.1, 0.3, 0.5, 0.7, 0.9, 1], 'min_child_weight': [1, 3, 5, 7, 9], 'max_depth': [3, 6, 9, 12, 15, 18, 21, 24], 'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ]), 'colsample_bytree': array([0.6, 0.7, 0.8, 0.9, 1. ]), 'reg_lambda': [0.01, 0.05, 0.1, 0.5, 1], 'reg_alpha': [0.01, 0.05, 0.1, 0.5, 1]}


In [7]:
### merge train and feature datasets
def merge_data(main, features):
    features_weHoliday = features.drop(["IsHoliday"], axis = 1)
    inputData = pd.merge(main, features_weHoliday, how = "left", on = ["Store", "Date"])
    return inputData


In [8]:
### Data cleaning 
## remove nas 
## remove negative values 

def data_cleaning(data):
    
    ##remove nas
    data = data.fillna(0)
    
    ## remove negative values
    data = data[data["Weekly_Sales"] >= 0]
    
    return data


In [9]:
## add year month date
def add_year_month_week(data):
    data["year"], data["month"], data["week"] = [pd.DatetimeIndex(data["Date"]).year, 
                                             pd.DatetimeIndex(data["Date"]).month,
                                             pd.DatetimeIndex(data["Date"]).weekofyear]
    data["month_str"] = ["0" + x if len(x) == 1 else x for x in data["month"].astype(str)]
    
    return data
# print(inputData.head())
# print(inputData[inputData["Week"] == 5].head())

In [10]:
## add trend function
def include_trend(data):
    label_encoder = LabelEncoder()
    data["week_trend"] = label_encoder.fit_transform(data["Date"]) + 1
    data["year_month"] = data["year"].astype(str) + "-" + data["month_str"].astype(str)
    data["month_trend"] = label_encoder.fit_transform(data["year_month"]) + 1
    data["year_trend"] = label_encoder.fit_transform(data["year"]) + 1
    
    return data

In [11]:
### add seasonality function
def include_seasonality(data, seasonality_type):
    
    ## initializing binarizer
    binarizer = LabelBinarizer()
    
    ## weekly seasonality
    week_encoded = binarizer.fit_transform(data["week"])
    week_encoded = np.delete(week_encoded, -1, 1)
    column_weeks = np.array(["week" + "_" + str(x) for x in binarizer.classes_[:len(binarizer.classes_)-1]])
    week_encoded = pd.DataFrame(week_encoded,
                               columns = column_weeks)
    
    
    ## monthly seasonality
    month_encoded = binarizer.fit_transform(data["month"])
    month_encoded = np.delete(month_encoded, -1, 1)
    column_months = np.array(["month" + "_" + str(x) for x in binarizer.classes_[:len(binarizer.classes_)-1]])
    month_encoded = pd.DataFrame(month_encoded,
                                    columns = column_months)
    
    if seasonality_type == "week":
        return pd.concat([data.reset_index(), week_encoded.reset_index()], axis = 1)
    
    else:
        return pd.concat([data.reset_index(), month_encoded.reset_index()], axis = 1)


# for checking 
# check = include_seasonality(train)

In [12]:
# add event/spike flags 
def add_event_flags(data, event_weeks):
    data = data.drop(["IsHoliday"], axis = 1)
    event_weeks = pd.DatetimeIndex(event_weeks)
    for week in event_weeks:
        data[str(week.year) + "_" + str(week.weekofyear) + "flag"] = [1 if x == week.date() else 0 for x in data["Date"]]
        
    return data
        



In [35]:
def model_train(model_name, x_train, y_train, x_val, y_val, ridge_range, lasso_range, random_forest_grid,
               hyperparameter_tuning = True):
    
    ## linear regression
    if model_name == "linear":
        model = LinearRegression()
        
        results = model.fit(x_train, y_train)
        train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
        val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
        min_wape_alpha = 0
        
        
    ## ridge regression    
    elif model_name == "ridge":
        accuracy_dict = {}
        for alpha in np.arange(ridge_range[0], ridge_range[1], 0.25):
            model = Ridge(alpha = alpha)
            results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            y_val_df = pd.DataFrame(y_val, columns = ["Weekly_Sales"])
            val_comparison = pd.concat([y_val_df.reset_index(), val_predicted.reset_index()], axis = 1)
            WAPE = abs(val_comparison["predicted"] - val_comparison["Weekly_Sales"]).sum()/val_comparison["Weekly_Sales"].sum()
            accuracy_dict[alpha] = WAPE
#             print("WAPE for " + str(alpha) + " is " + str(WAPE))
        
        min_wape_alpha = min(accuracy_dict, key = lambda k:accuracy_dict[k])
        model = Ridge(alpha = alpha)
        results = model.fit(x_train, y_train)
        val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
        train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
        
        
    ## lasso regression    
    elif model_name == "lasso":
        accuracy_dict = {}
        for alpha in np.arange(lasso_range[0], lasso_range[1], 0.25):
            model = Lasso(alpha = alpha)
            results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            y_val_df = pd.DataFrame(y_val, columns = ["Weekly_Sales"])
            val_comparison = pd.concat([y_val_df.reset_index(), val_predicted.reset_index()], axis = 1)
            WAPE = abs(val_comparison["predicted"] - val_comparison["Weekly_Sales"]).sum()/val_comparison["Weekly_Sales"].sum()
            accuracy_dict[alpha] = WAPE
#             print("WAPE for " + str(alpha) + " is " + str(WAPE))
        
        min_wape_alpha = min(accuracy_dict, key = lambda k:accuracy_dict[k])
        model = Ridge(alpha = alpha)
        results = model.fit(x_train, y_train)
        val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
        train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
        
        
    ## random forests    
    elif model_name == "random forest":
        if hyperparameter_tuning == True:
            x_all = np.append(x_train, x_val, axis = 0)
            y_all = np.append(y_train, y_val, axis = 0)
            tscv = TimeSeriesSplit(n_splits = 3)
            random_forest = RandomForestRegressor()
            model = RandomizedSearchCV(estimator = random_forest, param_distributions = random_forest_grid,
                                      n_iter = 100, cv = tscv, random_state = 42)
            results = model.fit(x_all, y_all)
    #         model = RandomForestRegressor(n_estimators = 100)
    #         results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
            min_wape_alpha = 0
#         {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 90}    
        else:
            random_forest = RandomForestRegressor()
            model = RandomForestRegressor(n_estimators = 400, min_samples_split = 2, min_samples_leaf = 1,
                                         max_features = "sqrt", max_depth = 90)
            results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
            min_wape_alpha = 0
        
    
    ## gradient boosting
    elif model_name == "gradient boosting":
        if hyperparameter_tuning == True:
            x_all = np.append(x_train, x_val, axis = 0)
            y_all = np.append(y_train, y_val, axis = 0)
            tscv = TimeSeriesSplit(n_splits = 3)
            gradient_boost = GradientBoostingRegressor()
            model = RandomizedSearchCV(estimator = gradient_boost, param_distributions = grid,
                                      n_iter = 100, cv = tscv, random_state = 42)
            results = model.fit(x_all, y_all)
    #         model = RandomForestRegressor(n_estimators = 100)
    #         results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
            min_wape_alpha = 0
#         {'n_estimators': 400, 'min_samples_split': 50, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 100, 'learning_rate': 0.18}
        else:
            model = GradientBoostingRegressor(n_estimators = 400, learning_rate = 0.18,
                                              min_samples_split = 50, min_samples_leaf = 1,
                                         max_features = "sqrt", max_depth = 100)
            results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
            min_wape_alpha = 0
            
            
    elif model_name == "xgboost":
        if hyperparameter_tuning == True:
            x_all = np.append(x_train, x_val, axis = 0)
            y_all = np.append(y_train, y_val, axis = 0)
            tscv = TimeSeriesSplit(n_splits = 3)
            xgb = XGBRegressor(objective = "reg:squarederror")
#             model = RandomizedSearchCV(estimator = xgb, param_distributions = grid,
#                                        cv = tscv, random_state = 42)
            model = GridSearchCV(estimator = xgb, param_grid = grid,
                                       cv = tscv)
            results = model.fit(x_all, y_all)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
            min_wape_alpha = 0
#         'subsample': 1.0, 'reg_lambda': 0.05, 'reg_alpha': 0.01, 'n_estimators': 1200, 'min_child_weight': 1, 'max_depth': 3, 'gamma': 0.3, 'eta': 0.015, 'colsample_bytree': 0.9
        else:
            model = xg.XGBRegressor(objective = "reg:squarederror",
                                    subsample = 1,
                                    n_estimators = 1200,
                                   reg_lambda = 0.05,
                                   reg_alpha = 0.01,
                                   min_child_weight = 1,
                                   max_depth = 3,
                                   gamma = 0.3,
                                   eta = 0.015,
                                   colsample_bytree = 1)
            results = model.fit(x_train, y_train)
            val_predicted = pd.DataFrame(results.predict(x_val), columns = ["predicted"])
            train_predicted = pd.DataFrame(results.predict(x_train), columns = ["predicted"])
            min_wape_alpha = 0
    
        
        
    
    else:
        return "Error"
    
    return train_predicted, val_predicted, min_wape_alpha, results
            
        

    


In [15]:
### calculate error metrics - rmse, wape

def error_metrics(data):
    
    ##wape
    data["absolute_error"] = abs(data["predicted"] - data["Weekly_Sales"])
    WAPE = data["absolute_error"].sum()/data["Weekly_Sales"].sum()
    data["WAPE"] = WAPE
    
    ##rmse
    RMSE = np.sqrt(np.square(data["absolute_error"]).mean())
    data["RMSE"] = RMSE
    
    ##wmae
    WMAE = np.sum(data["absolute_error"] * data["Weekly_Sales"])/np.sum(data["Weekly_Sales"])
    data["WMAE"] = WMAE
    
    return data

## f
# check_data, wape, rmse, wmae = error_metrics(trained_data)
    


In [29]:
def model_run(data, level, model_name, ridge_range, lasso_range, random_forest_grid,
              test_run = True, validation_split = 0.2):
    
    
    columns_to_drop = ["Store", "Dept", "Date", "month_str", "year_month", "week_trend", "year_trend"]
    target = "Weekly_Sales"
    full_dataset = pd.DataFrame()
    i = 0
    
    if test_run == True:
        check_data = data[(data["Store"] < 2 )  & (data["Dept"] < 2)]
        grouped = check_data.groupby(["Store", "Dept"])
    else:
        grouped = data.groupby(["Store", "Dept"])
        
    for name, group in grouped:
        temp_data = data[(data["Store"] == name[0]) & (data["Dept"] == name[1])]
        print(name[0], name[1])
        if len(temp_data) >= 52:
            y_dataframe = pd.DataFrame(temp_data[target])
            y_columns = y_dataframe.columns 
            y = np.array(y_dataframe)

            temp_data = temp_data.drop(target, axis = 1)
            x_dropped = temp_data[columns_to_drop]
            x_dataframe = temp_data.drop(columns_to_drop, axis = 1)
            x_columns = x_dataframe.columns
            x = np.array(x_dataframe)


            ## train, validation split
            x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = validation_split)
            
            train_predicted, val_predicted, min_wape_alpha, model = model_train(model_name, x_train, y_train, x_val, y_val, ridge_range, lasso_range, random_forest_grid)


            print(np.mean(train_predicted["predicted"]))
            print(np.mean(val_predicted["predicted"]))
            
            x_train_df = pd.DataFrame(x_train, columns = x_columns)
            y_train_df = pd.DataFrame(y_train, columns = y_columns)
            x_val_df = pd.DataFrame(x_val, columns = x_columns)
            y_val_df = pd.DataFrame(y_val, columns = y_columns)

            train_wpredicted = pd.concat([x_train_df, y_train_df, train_predicted], axis = 1)
            train_wpredicted["tag"] = "train"
            train_wpredicted = error_metrics(train_wpredicted)
            
            val_wpredicted = pd.concat([x_val_df, y_val_df, val_predicted], axis = 1)
            val_wpredicted["tag"] = "validation"
            val_wpredicted = error_metrics(val_wpredicted)
            print(np.mean(val_wpredicted["WAPE"]))
            train_and_val = pd.concat([train_wpredicted, val_wpredicted])
            train_and_val_wdropped = pd.concat([train_and_val.reset_index(), x_dropped.reset_index()], axis = 1)

            full_dataset = full_dataset.append(train_and_val_wdropped)
            i += 1
            print(i , " iterations complete")
        
    return full_dataset, model

In [17]:
def model_run(data, level, test_run = True, validation_split = 0.2, ridge_range, lasso_range):
    
    model = LinearRegression()
    
    columns_to_drop = ["Store", "Dept", "Date", "month_str", "year_month", "week_trend", "year_trend"]
    target = "Weekly_Sales"
    full_dataset = pd.DataFrame()
    i = 0
    
    if test_run == True:
        check_data = data[(data["Store"] < 4 )  & (data["Dept"] < 4)]
        grouped = check_data.groupby(["Store", "Dept"])
    else:
        grouped = data.groupby(["Store", "Dept"])
        
    for name, group in grouped:
        temp_data = data[(data["Store"] == name[0]) & (data["Dept"] == name[1])]
        print(name[0], name[1])
        if len(temp_data) >= 52:
            y_dataframe = pd.DataFrame(temp_data[target])
            y_columns = y_dataframe.columns 
            y = np.array(y_dataframe)

            temp_data = temp_data.drop(target, axis = 1)
            x_dropped = temp_data[columns_to_drop]
            x_dataframe = temp_data.drop(columns_to_drop, axis = 1)
            x_columns = x_dataframe.columns
            x = np.array(x_dataframe)


            ## train, validation split
            x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = validation_split)


            x_train_wconstant = sm.add_constant(x_train)
            x_val_wconstant = sm.add_constant(x_val)

            model = sm.OLS(y_train, x_train_wconstant)

            results = model.fit()

            train_predicted = pd.DataFrame(results.fittedvalues, columns = ["predicted"])
            print(x_val_wconstant.shape)
            print(results.predict(x_val_wconstant).shape)
            val_predicted = pd.DataFrame(results.predict(x_val_wconstant), columns = ["predicted"])

            x_train_df = pd.DataFrame(x_train, columns = x_columns)
            y_train_df = pd.DataFrame(y_train, columns = y_columns)
            x_val_df = pd.DataFrame(x_val, columns = x_columns)
            y_val_df = pd.DataFrame(y_val, columns = y_columns)

            train_wpredicted = pd.concat([x_train_df, y_train_df, train_predicted], axis = 1)
            train_wpredicted["tag"] = "train"
            train_wpredicted = error_metrics(train_wpredicted)
            
            val_wpredicted = pd.concat([x_val_df, y_val_df, val_predicted], axis = 1)
            val_wpredicted["tag"] = "validation"
            val_wpredicted = error_metrics(val_wpredicted)

            train_and_val = pd.concat([train_wpredicted, val_wpredicted])
            train_and_val_wdropped = pd.concat([train_and_val.reset_index(), x_dropped.reset_index()], axis = 1)

            full_dataset = full_dataset.append(train_and_val_wdropped)
            i += 1
            print(i , " iterations complete")
        
    return full_dataset

SyntaxError: non-default argument follows default argument (<ipython-input-17-fe8f80f791ac>, line 1)

In [ ]:
if __name__ == "__main__":
    input_data = merge_data(train, features)
    input_data = data_cleaning(input_data)
    input_data = add_year_month_week(input_data)
    input_data = include_trend(input_data)
    input_data = include_seasonality(input_data,
                                           seasonality_type)
    input_data = add_event_flags(input_data, event_weeks)
    
    input_data.to_csv("datasets/processeddatasets/input_data.csv")
    
    
    trained_data, model = model_run(input_data, level, model_name, ridge_range, lasso_range, grid, test_run = True)
    
#     trained_data.to_csv("datasets/processeddatasets/trained_data.csv")

    

1 1


In [27]:
# model_filename = "testmodel_gb.sav"
# pickle.dump(model, open(model_filename, 'wb'))
print(np.mean(trained_data[trained_data["tag"] == "train"]["WAPE"]))
print(np.mean(trained_data[trained_data["tag"] == "validation"]["WAPE"]))
print(np.mean(trained_data[trained_data["tag"] == "train"]["WMAE"]))
print(np.mean(trained_data[trained_data["tag"] == "validation"]["WMAE"]))

# default random forest settings
# train wape - 0.054124083923314886
# val wape - 0.16467224121602228

## tuned random forest 
# train wape - 0.054418925093969624
# val wape - 0.14835607061222403


0.01716813790581898
0.14336897473748045
374.65554936248316
4132.081014542145
